# CPD Data Visualization

---
- Date: 11/26/2019
- Author: Simon (Hyungjun) Park


In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import copy
from zipfile import ZipFile
from pathlib import PurePath

## Unzip and read CSV files

In [2]:
# Set the path
git_path = PurePath(os.getcwd())
path = git_path.parents[0]

In [3]:
# Prepare CSV file names in the zip file, the zip file path, and csv file path(s)
cpd_path = os.path.join(path, 'chicago-police-data', 'data')
zip_name = [c for c in os.listdir(cpd_path) if c.endswith('.zip')]
zip_path = os.path.join(cpd_path, zip_name[0])
zip_names = ['/complaints-accused_2000-2016_2016-11.csv.gz',
            '/complaints-victims_2000-2016_2016-11.csv.gz',
            '/complaints-investigators_2000-2016_2016-11.csv.gz',
            '/complaints-complaints_2000-2018_2018-03.csv.gz',
            '/roster_1936-2017_2017-04.csv.gz']
dis_path = os.path.join(cpd_path, 'context_data/discipline_penalty_codes.csv')

In [4]:
# UDF that deals with Unzip and pd.read_csv
def read_in_zip_csv(zippath, csvpath, dtype = None):
    """
    Read a single csv file inside a zipfile and returns a pandas data frame.
    If a provided csv file path does not have a unique match in the zipfile, returns warning.
    The following modules are necessary for this function.
    - import pandas as pd
    - from zipfile import ZipFile
    
    Input
    --------
    zippath: the absolute path for a zip file
    
    csvpath: the directory of a csv file within the zip file.
        It matches with '.endswith', therefore only need to provide the ending directory.
    
    dtype: if the csv file requries dtype to be specified.
        see pandas .read_csv for details.
    
    Return
    -------
    If there is a unique match:
        A dataframe
    
    If there is no match:
        A warning
    
    If there is multiple match:
        A warning and match values

    """
    with ZipFile(zippath) as zf:
        print('From', zippath)
        csv = [c for c in zf.namelist() if c.endswith(csvpath)]
        if len(csv)>1: 
            print("  Multiple match: check the csv file path within the zip file and provide a unique path.")
            for i, f in enumerate(csv): print('Match', i, ':', f)
            return()
        elif len(csv)<1:
            print("  No match: check the csv file path within the zip file and provide a unique path.")
            return()
        else:
            print('  Loading', csv)
            if pd.isnull(dtype):
                df = pd.read_csv(zf.open(csv[0]), compression='gzip')
            else:
                df = pd.read_csv(zf.open(csv[0]), compression='gzip', dtype=dtype)
    return(df)

In [5]:
# Read CSV files
df_acc = pd.read_csv(os.getcwd() + '/complaints-accused_2000-2016_2016-11.csv.gz')
df_vic = pd.read_csv(os.getcwd() + '/complaints-victims_2000-2016_2016-11.csv.gz')
df_inv = pd.read_csv(os.getcwd() + '/complaints-investigators_2000-2016_2016-11.csv.gz')
df_cmpl = pd.read_csv(os.getcwd() + '/complaints-complaints_2000-2018_2018-03.csv.gz')
df_roster = pd.read_csv(os.getcwd() + '/roster_1936-2017_2017-04.csv')
df_dis = pd.read_csv(os.getcwd() + '/discipline_penalty_codes.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# cleanup df_roaster
df_roster = df_roster.drop(columns = ['row_id'])

In [7]:
# cleanup df_acc
df_acc = df_acc.drop(columns = ['row_id'])\
               .merge(df_roster,
                      on = ['UID','old_UID', 'link_UID'], how = 'left') \
               .merge(df_dis,
                      left_on ='recommended_discipline', 
                      right_on = 'CODE' ,how = 'left').drop(columns = ['CODE','NOTES'])\
               .merge(df_dis, 
                      left_on ='final_discipline', 
                      right_on = 'CODE', how = 'left').drop(columns = ['CODE','NOTES'])\
               .rename(columns={'complaints-accused_2000-2016_2016-11_ID': 'complaints-accused_id',
                              'recommended_discipline': 'REC_DSPLN_code',
                              'final_discipline': 'FIN_DSPLN_code',
                              'recommended_finding': 'REC_finding',
                              'final_finding': 'FIN_finding',
                              'ACTION_TAKEN_x': 'REC_DSPLN',
                              'ACTION_TAKEN_y': 'FIN_DSPLN'})

In [8]:
# cleanup df_inv
df_inv = df_inv.drop(columns = ['row_id'])\
               .rename(columns = {'complaints-investigators_2000-2016_2016-11_ID': 'complaints-investigators_id'})

In [9]:
# cleanup df_vic
df_vic = df_vic.rename(columns={'gender':'gender_vic',
                              'age': 'age_vic',
                              'race': 'race_vic'})

In [10]:
# cleanup df_cmpl
df_cmpl = df_cmpl.drop(columns = ['row_id']) \
                 .assign(complaint_year  = lambda df: pd.to_datetime(df.complaint_date).dt.year.astype(str)) \
                 .assign(complaint_month = lambda df: pd.to_datetime(df.complaint_date).dt.month.astype(str))\
                 .assign(complaint_year_month = lambda df: df.complaint_year.str.cat(df.complaint_month, sep='/'))

In [11]:
# merge all datasets
df_all = df_acc.merge(df_inv, on = 'cr_id', how = 'left',suffixes=('_acc', '_inv')) \
               .merge(df_vic, on = 'cr_id', how = 'left') \
               .merge(df_cmpl, on = 'cr_id', how = 'left') \

## Interactive Data Visualization via Bokeh

**Note**: The following codes are written based on Bokeh version 1.3.4.  [The latest version of Bokeh (1.4.0)](https://docs.bokeh.org/en/latest/docs/releases.html), which was release October 2019, deprecated `legend` keyword argument and  replaced it with `legend_label`, `legend_field`, `legend_group`. If you use version 1.4.0, some warning messages may appear due to the version conflict. However, this warning doesn't matter our results or if the warning annoys you, please replace our `legend` argument with `legend_label` (We ensured the whole codes works with `legend_label` under version 1.4.0).

In [12]:
import bokeh
from ipywidgets import interact, interact_manual
from bokeh.models.widgets import Panel, Tabs
from bokeh.transform import cumsum
from bokeh.plotting import figure, show
from bokeh.palettes import Category20c
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from math import pi
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [13]:
def cpd_analyzer(df, key = 'cr_id', field = ['REC_finding', 'FIN_finding'], group = None, prop = True):
    """
    Calculate the proportions for interesting category
    In the processing, remove duplicated observations depending on
    finding categories and other subsetting variables
    
    Input
    --------
    df: dataframe
        Merged CPD dataset
    
    key: str (default: 'cr_id')
        Variable you want to count in specific field
    
    field: List of strings (default: ['REC_finding', 'FIN_finding'])
        List of category which you want to count the number of key observation
        and calculate those proportions.
        Note: Those should be the same as the part of columns names in df
        
    group: List of strings(default: None)
        List of strings you want to make groupings 
        
    
    prop: Bool (default: True)
        True is to calculate the proportion (percentage value)
    
    Output
    -------
    dataframe
    
    """
    output = pd.DataFrame(index = [])
    if group == None:
        for f in field:      
            df_subset = df[[key, f]]
            df_subset = df_subset[~df_subset.duplicated(keep = 'first')]

            result = df_subset.groupby(f).count()
            if prop:
                result = result.apply(lambda x:x/x.sum()*100, axis = 0)
                result = result.rename(columns = {key:'{}(%)'.format(f)})
                output = pd.concat([output, result], axis=1, sort=False)
            else:
                result = result.rename(columns = {key:'{}(N)'.format(f)})
                output = pd.concat([output, result], axis=1, sort=False)
    else:
        if field == None:
            subset = [key] + group
            df_subset = df[subset]
            df_subset = df_subset[~df_subset.duplicated(keep = 'first')]
            
            result = df_subset.groupby(group).count()
            output = pd.concat([output, result], axis=1, sort=False)
        else:    
             for f in field:
                subset = [key, f] + group
                df_subset = df[subset]
                df_subset = df_subset[~df_subset.duplicated(keep = 'first')]

                group_by = [f] + group
                result = df_subset.groupby(group_by).count()
                if prop:
                    result = result.groupby(group).apply(lambda x:x/x.sum()*100)
                    result = result.rename(columns = {key:'{}(%)'.format(f)})
                    output = pd.concat([output, result], axis=1, sort=False)
                else:
                    result = result.rename(columns = {key:'{}(N)'.format(f)})
                    output = pd.concat([output, result], axis=1, sort=False)
    return output

In [14]:
# useful function to reconfigure the data
def df_reconfig(df, level = 0):
    """
    Description

    input
    -----
    multilevel indexed dataframe
    output
    -----
    reconfigured dataframe
    """
    data = df.unstack(level = level)
    data.columns = data.columns.droplevel()
    data = data.reset_index()
    return data

### Line plot: Change of Number of Complaints by Outcomes

* Dropdown Menu
* Interactive Legends (Click)

In [15]:
df_ts = cpd_analyzer(df_all, group = ['complaint_year'], prop = False)

In [16]:
def cpd_lineplot(df, outcome):
    data = df.loc[outcome]
    
    plot = figure(title = outcome, x_axis_label = 'Year', y_axis_label = 'Number of Incidents', plot_height = 300)
    plot.line(data.index, data['REC_finding(N)'], legend = 'Recommend', line_width = 4)
    plot.line(data.index, data['FIN_finding(N)'], legend = 'Final', color = 'orange', line_width = 4)
    plot.legend.click_policy="hide"
    
    return plot

In [17]:
findings = ['SU', 'NAF', 'NS', 'EX', 'UN']
@interact(Outcome = findings)
def dropdown_click(Outcome = findings[0]):
    plot = cpd_lineplot(df_ts, Outcome)
    show(plot)

interactive(children=(Dropdown(description='Outcome', options=('SU', 'NAF', 'NS', 'EX', 'UN'), value='SU'), Ou…

### Bar plot: Change of Number and Proportion of Accused Officers

* Tabs

In [18]:
def color_gen(n):
    palette = bokeh.palettes.Category20c[n]
    for color in palette:
        yield color

In [19]:
df_offc_race_num = cpd_analyzer(df_all, group = ['complaint_year'], field = ['race'], prop = False)
df_offc_race_prop = cpd_analyzer(df_all, group = ['complaint_year'], field = ['race'], prop = True)

In [20]:
df_offc_race_num = df_reconfig(df_offc_race_num) \
    .assign(OTHERS = lambda df:df['ASIAN/PACIFIC ISLANDER'] + df['NATIVE AMERICAN/ALASKAN NATIVE'])\
    .drop(columns = ['ASIAN/PACIFIC ISLANDER', 'NATIVE AMERICAN/ALASKAN NATIVE'])
df_offc_race_prop = df_reconfig(df_offc_race_prop) \
    .assign(OTHERS = lambda df:df['ASIAN/PACIFIC ISLANDER'] + df['NATIVE AMERICAN/ALASKAN NATIVE'])\
    .drop(columns = ['ASIAN/PACIFIC ISLANDER', 'NATIVE AMERICAN/ALASKAN NATIVE'])

In [21]:
def cpd_stackbar_tab(df_num, df_prop, groups):
    colors = [i for i in color_gen(len(groups))]
    names = [i.lower() for i in groups]
    
    tbs = []
    for i, (l, data) in enumerate(zip(['Number', 'Proportion'],[df_num, df_prop])):       
        plot = figure(title='Accused Officers by Race',
                      x_axis_label='Year', y_axis_label='The number of Accused Officers',
                      x_range=data['complaint_year'], plot_height = 300)
        plot.vbar_stack(groups, x = 'complaint_year', width=0.9, color = colors, source = data, legend = names)
        plot.y_range.start = 0
        plot.x_range.range_padding = 0.1
        plot.xgrid.grid_line_color = None
        plot.axis.minor_tick_line_color = None
        plot.outline_line_color = None
        plot.legend.location = "top_right"
        plot.legend.orientation = "horizontal"
        
        tbs.append(Panel(child = plot, title = '{}'.format(l)))
    
    tabs = Tabs(tabs = tbs)
    
    return tabs

In [22]:
races = ['WHITE', 'BLACK', 'HISPANIC', 'OTHERS']
tabs = cpd_stackbar_tab(df_offc_race_num, df_offc_race_prop, groups = races)
show(tabs)

### Histogram: Distribution of Accusation

* Slider Option

In [23]:
df_num_acc = cpd_analyzer(df_all, group = ['race'], field = ['UID_acc'], prop = False)

In [24]:
def cpd_binning(df, binwidth):
    df = df.groupby(['race', pd.cut(df['UID_acc(N)'], bins = [i for i in range(0,60, binwidth)])]).count()
    df = df_reconfig(df)
    df['UID_acc(N)'] = df['UID_acc(N)'].astype(str)
    df = df \
    .assign(OTHERS = lambda df:df['ASIAN/PACIFIC ISLANDER'] + df['NATIVE AMERICAN/ALASKAN NATIVE'])\
    .drop(columns = ['ASIAN/PACIFIC ISLANDER', 'NATIVE AMERICAN/ALASKAN NATIVE'])
    
    return(df)

In [25]:
def cpd_stackbar_bin(df, groups):
    colors = [i for i in color_gen(len(groups))]
    names = [i.lower() for i in groups]

    plot = figure(title='Distribution of Accusation',
                  x_axis_label='Total Accusation', y_axis_label='Officers', 
                  x_range=df['UID_acc(N)'], plot_height = 300)
    plot.vbar_stack(groups, x = 'UID_acc(N)', width=0.8, color = colors, source = df, legend = names)
    plot.y_range.start = 0
    plot.x_range.range_padding = 0.1
    plot.xgrid.grid_line_color = None
    plot.axis.minor_tick_line_color = None
    plot.outline_line_color = None
    plot.legend.location = "top_right"
    plot.legend.orientation = "horizontal"
    plot.xaxis.major_label_orientation = 1.2
        
    return plot

In [26]:
@interact(Bins=(2, 20))
def slider(Bins=4):
    df = cpd_binning(df_num_acc, Bins)
    plot = cpd_stackbar_bin(df, groups = races)
    show(plot)

interactive(children=(IntSlider(value=4, description='Bins', max=20, min=2), Output()), _dom_classes=('widget-…

### Multiple widgets: Complaints Category and Incident Location Ranking

* Dropdown Menu
* Slider Bar
* Checkbox
* Tabs
* Hover over the Pie Chart

In [27]:
df_comtype = cpd_analyzer(df_all, group = ['complainant_type','incident_location'],
                          field = ['current_complaint_category'], prop = False)

In [28]:
df_comtype = df_reconfig(df_comtype, level = 1)

In [29]:
def cpd_rankplot_pie_tab(df, types, rank, prop = False):
    title = ['Complaints', 'Locations']
    kinds = ['current_complaint_category', 'incident_location']
    bywho = [types]
    
    tbs = []
    for title, kind in zip(title, kinds):
        df = df_comtype.groupby(kind).sum()
        plt = []
        for who in bywho:
            df_sorted = df.sort_values(who)
            xlab = 'Number of Complaints'
           
            if prop: 
                df_sorted = df_sorted.apply(lambda x:x/x.sum()*100)
                xlab = 'Percentage of Complaints'
                
            df_ranked = copy.deepcopy(df_sorted.tail(rank)[::-1])
            df_ranked.loc['RANK {}~{}'.format(rank+1,len(df_sorted)),:] = df_sorted.head(len(df_sorted)-rank).sum(axis = 0)
            df_ranked = df_ranked.reset_index()
            df_ranked.columns.rename(None, inplace=True)
            df_ranked.index = np.arange(1, len(df_ranked) + 1)
            df_ranked['angle'] = df_ranked[who]/df_ranked[who].sum() * 2*pi
            df_ranked['color'] = Category20c[len(df_ranked)]
            df_ranked = df_ranked[::-1]
            df_ranked['color'][len(df_ranked)]='#d9d9d9'
                        
            yrange = [i for i in df_ranked[kind]]
            plot1 = figure(title = 'Top {} {} by {}'.format(rank, title, who), 
                           x_axis_label = xlab, y_axis_label = 'Category',
                           y_range = yrange, plot_height = 300)
            plot1.hbar(y = kind, right = who, height = 0.8,
                       line_color="white", fill_color='color', 
                       source = df_ranked)
            plot1.ygrid.grid_line_color = None
            plot1.x_range.start = 0
            plt.append(plot1)

            
            plot2 = figure(plot_height=300, title = 'Pie Chart',
                           toolbar_location=None,
                           tools="hover", tooltips="@{}: @{}".format(kind, who), x_range=(-0.5, 1.0))
            plot2.wedge(x=0, y=1, radius=0.28,
                        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                        line_color="white", fill_color='color', source=df_ranked)
            plot2.axis.axis_label=None
            plot2.axis.visible=False
            plot2.grid.grid_line_color = None
            plt.append(plot2)

        plots = gridplot([plt])
        tbs.append(Panel(child = plots, title = '{}'.format(title)))
   
    tabs = Tabs(tabs = tbs)
    return tabs

In [30]:
types = ['CIVILIAN','CPD_EMPLOYEE']
@interact_manual(Type = types, Rank=(3, 19), Percentage = False)
def dropdown_slider_checkbox_tab(Type = types[0], Rank = 19, Percentage = False):
    tabs = cpd_rankplot_pie_tab(df_comtype, Type, Rank, Percentage)
    show(tabs)

interactive(children=(Dropdown(description='Type', options=('CIVILIAN', 'CPD_EMPLOYEE'), value='CIVILIAN'), In…